In [1]:
# Using CNN with the Cats vs Dogs Dataset

import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [2]:
# skidamo dataset (kao zip fajl) u tmp folderu, a zatim ga otkapujemo u tom istom folderu (Pet images)
!wget --no-check-certificate \
    "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip" \
    -O "/tmp/cats-and-dogs.zip"

local_zip = '/tmp/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

--2023-03-06 23:03:30--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.51.99.68, 2600:1413:1:597::317f, 2600:1413:1:5a0::317f
Connecting to download.microsoft.com (download.microsoft.com)|184.51.99.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.67M   228MB/s    in 3.4s    

2023-03-06 23:03:34 (234 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824887076/824887076]



In [3]:
# kreiramo funckiju koja ce kreirati neophodne direktorijume
def create_train_val_dirs(root_path):

  base_dir = root_path
  os.makedirs(root_path)

  training_dir = os.path.join(base_dir, 'training')
  os.makedirs(training_dir)

  validation_dir = os.path.join(base_dir, 'validation')
  os.makedirs(validation_dir)
  
  training_cats_dir = os.path.join(training_dir, 'cats')
  os.makedirs(training_cats_dir)

  training_dogs_dir = os.path.join(training_dir, 'dogs')
  os.makedirs(training_dogs_dir)

  validation_cats_dir = os.path.join(validation_dir, 'cats')
  os.makedirs(validation_cats_dir)

  validation_dogs_dir = os.path.join(validation_dir, 'dogs')
  os.makedirs(validation_dogs_dir)

In [4]:
# definisemo funkciju koja ce podeliti podatke za trening i validaciju
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  
  # izvlacimo podatke iz SOURCE_DIR
  sd = os.listdir(SOURCE_DIR)
  # brisemo sve podatke cija je velicina jednaka 0
  for fname in sd:
    if os.path.getsize(os.path.join(SOURCE_DIR, fname)) <= 0:
      print(fname, "is zero length, so ignoring.")
      sd.remove(fname)
  
  # definisemo velicinu training i validation seta
  training_size = round(len(sd) * SPLIT_SIZE)
  validation_size = round(len(sd) * (1 - SPLIT_SIZE))

  # shuffle-ujemo podatke 
  random.sample(sd, len(sd))

  # upisujemo podatke u training i validation sample
  training_sample = random.sample(sd[:training_size], training_size)
  validation_sample = random.sample(sd[training_size:], validation_size)

  # upisujemo podatke u odgovarajuce direktorijume
  for fname in training_sample:
    copyfile(os.path.join(SOURCE_DIR, fname), os.path.join(TRAINING_DIR, fname))

  for fname in validation_sample:
    copyfile(os.path.join(SOURCE_DIR, fname), os.path.join(VALIDATION_DIR, fname))

In [5]:
# kreiramo training i validation data generatore
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  
  # instanciramo ImageDataGenerator klasu
  train_datagen = ImageDataGenerator(rescale = 1./255)

  # kreiramo training generator
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=10,
                                                      class_mode='binary',
                                                      target_size=(150, 150))
  
  # instanciramo ImageGenerator klasu za validation data
  validation_datagen = ImageDataGenerator(rescale = 1./255)

  # kreiramo validation generator
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=10,
                                                                class_mode='binary',
                                                                target_size=(150, 150))
  
  return train_generator, validation_generator

In [6]:
# kreiramo model

from tensorflow.keras.optimizers import RMSprop
def create_model():

  model = tf.keras.models.Sequential([ 
      # kreiramo prvi Convolution layer
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
      tf.keras.layers.MaxPooling2D(2,2),
      
      # kreiramo drugi Convolution layer
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),

      # kreiramo treci Convolution layer
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),

      # pretvaramo matricu u 1-dimenzionalni niz
      tf.keras.layers.Flatten(),
      # definisemo sloj sa 512 neurona i aktivacionom funkcijom relu
      tf.keras.layers.Dense(512, activation='relu'),
      # definisemo 1 izlazni neuron sa aktivacioniom funkcijom sigmoid
      tf.keras.layers.Dense(1, activation='sigmoid'),
  ])


  model.compile(optimizer=RMSprop(learning_rate=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy']) 

  return model

In [7]:
# definisemo root direktorijum
root_dir = '/tmp/cats-v-dogs'

# ako podaci u direktorijumu vec postoje, obrisi ih (korisceno za potrebe testiranja)
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)
  
# pozivamo funkciju za kreiranje direktorijuma
create_train_val_dirs(root_path=root_dir)

# definisemo putanje 
CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DIR = "/tmp/cats-v-dogs/training/"
VALIDATION_DIR = "/tmp/cats-v-dogs/validation/"
TRAINING_CATS_DIR = os.path.join(TRAINING_DIR, "cats/")
VALIDATION_CATS_DIR = os.path.join(VALIDATION_DIR, "cats/")
TRAINING_DOGS_DIR = os.path.join(TRAINING_DIR, "dogs/")
VALIDATION_DOGS_DIR = os.path.join(VALIDATION_DIR, "dogs/")

# definisemo split_size
split_size = 0.9

In [25]:
# pozivamo funkciju za podelu podataka
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, VALIDATION_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, VALIDATION_DOGS_DIR, split_size)

# pozivamo funkciju za kreiranje generatora
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

# kreiramo model
model = create_model()

# treniramo model
history = model.fit(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=validation_generator)



666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.
Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Epoch 1/10
1984/2250 [=========================>....] - ETA: 10s - loss: 0.6075 - accuracy: 0.6650

/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Truncated File Read
  warnings.warn(str(msg))


2250/2250 [==============================] - 97s 42ms/step - loss: 0.5981 - accuracy: 0.6737 - val_loss: 0.5229 - val_accuracy: 0.7364
Epoch 2/10
2250/2250 [==============================] - 106s 47ms/step - loss: 0.4770 - accuracy: 0.7799 - val_loss: 0.4026 - val_accuracy: 0.8208
Epoch 3/10
2250/2250 [==============================] - 95s 42ms/step - loss: 0.4050 - accuracy: 0.8216 - val_loss: 0.4734 - val_accuracy: 0.7928
Epoch 4/10
2250/2250 [==============================] - 97s 43ms/step - loss: 0.3391 - accuracy: 0.8593 - val_loss: 0.3851 - val_accuracy: 0.8332
Epoch 5/10
2250/2250 [==============================] - 97s 43ms/step - loss: 0.2793 - accuracy: 0.8890 - val_loss: 0.4223 - val_accuracy: 0.8324
Epoch 6/10
2250/2250 [==============================] - 96s 43ms/step - loss: 0.2207 - accuracy: 0.9185 - val_loss: 0.6594 - val_accuracy: 0.8016
Epoch 7/10
2250/2250 [==============================] - 96s 42ms/step - loss: 0.1770 - accuracy: 0.9400 - val_loss: 0.7880 - val_accur

In [30]:
# brisemo folder ukoliko vec postoji
shutil.rmtree('/tmp/saved_model')

# cuvamo model
os.makedirs('/tmp/saved_model')
model.save('/tmp/saved_model/my_model.h5')

In [18]:
# ucitavamo model
# model = tf.keras.models.load_model('/tmp/saved_model/my_model.h5')

In [31]:
# testiranje modela
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

# trazimo od korisnika da unese sliku
print('Dodajte sliku: ')
uploaded = files.upload()

# izvlacimo putanju unete slike
fn = list(uploaded.keys())[0]
path = '/content/' + str(fn)

# sredjujemo sliku tako da odgovora datasetu
img = load_img(path, target_size=(150,150))
x = img_to_array(img)
x = x / 255
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

# pozivamo model koji daje svoju predikciju
classes = model.predict(images, batch_size=10)
print(classes[0])

# ispisujemo rezultat 
if(classes[0]) > 0.5:
  print("Na slici koju ste uneli se nalazi pas.")
else:
  print("Na slici koju ste uneli se nalazi mačka.")


Dodajte sliku: 


Saving WhatsApp Image 2023-03-07 at 00.27.47.jpg to WhatsApp Image 2023-03-07 at 00.27.47 (1).jpg
1/1 [==============================] - 0s 111ms/step
[0.95087254]
Na slici koju ste uneli se nalazi pas.
